# Spark + Neo4j

To reset database: 
"MATCH (n) DETACH
DELETE n"

Nodes:
- Person
    - reviewerID
    - reviewerName
- Review
    - asin
    - helpful
    - reviewText
    - overall
    - summary
    - unixReviewTime
    - reviewTime
- Product
    - asin
    - title
    - price
    - imUrl
    - brand
- Category
    - categoryName

Relations:
- Person -[WRITTEN]- Review
- Person -[REVIEWED]- Product
- Review -[CONCERNED]- Product
- Product -[BOUGHTWITH]- Product   
- Product -[VIEWEDWITH]- Product
- Product -[BELONGING]- Category
- Product -[RANKING]- Category
    - salesRank

In [88]:
from neo4j.v1 import GraphDatabase, basic_auth

driver = GraphDatabase.driver("bolt://localhost", auth=basic_auth("neo4j", "comptools"))
session = driver.session()
session.run("CREATE (a:Person {name:'Bob'})")
result = session.run("MATCH (a:Person) RETURN a.name AS name")
for record in result:
    print(record["name"])
session.close()

Bob


# Very simple startup with Spark (WORKING!!!)

In [4]:
import ujson
import ast

def json_parser(line):
    review_dictionary = ast.literal_eval(line)
    if 'price' in review_dictionary.keys():
        
        #yield "reviewerID", review_dictionary['reviewerID']
        #yield "asin", review_dictionary['asin']
        #yield "reviewerName", review_dictionary['reviewerName']
        #yield "helpful", review_dictionary['helpful']
        #yield "reviewText", review_dictionary['reviewText']
        yield "price", review_dictionary['price']
        #yield "summary", review_dictionary['summary']
        #yield "unixReviewTime", review_dictionary['unixReviewTime']
        #yield "reviewTime", review_dictionary['reviewTime']
        yield 'count', 1
    
lines = sc.textFile("../../amazon_dataset/metadata.json")
review_values = lines.flatMap(json_parser)
key_lists = review_values.reduceByKey(lambda a,b: a+b)

a = key_lists.collect()

print a

print a[0][1]*1.0/a[1][1]

[('count', 6063208), ('price', 211831713.8699847)]
0.0286227585531


In [57]:
result = {}
for elm in a:
    if elm[0]=='count':
        result['count']=elm[1]
    else:
        result['price']=elm[1]

In [58]:
print 'The average price of all the products is:',result['price']/result['count']

The average price of all the products is: 34.9372335354


In [66]:
music = spark.read.json('../../amazon_dataset/reviews_Musical_Instruments_5.json')
music.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [76]:
music.registerTempTable('music')
result = spark.sql("SELECT asin as ProductID,summary,helpful,unixReviewTime FROM music WHERE overall==5.0 AND helpful[1]-helpful[0]<10 AND helpful[0]!=0 AND helpful[1]!=0 AND helpful[0]>100")
result.show(10, truncate=False)

+----------+------------------------------------------------------------+----------+--------------+
|ProductID |summary                                                     |helpful   |unixReviewTime|
+----------+------------------------------------------------------------+----------+--------------+
|B0002GXZK4|This guitar DOES have a BIG SOUND for a small price         |[156, 160]|1111708800    |
|B0002H0H4A|The Pop Rocks with the Yeti                                 |[101, 105]|1342656000    |
|B000B6DHB2|Impressive Sound from an Inexpensive Source - UPDATE 7/11/14|[130, 136]|1230422400    |
|B000BWEM42|Well built folding stand great for guitar music and tab     |[156, 161]|1197504000    |
|B002Q0WSO8|Finally!  Amazon offers the Intelli IMT-500                 |[188, 193]|1177027200    |
|B003FVB7XA|Considerably better than anticipated!                       |[259, 266]|1223683200    |
|B0042EZH6W|Bought it for my Kid. Now I'm buying it for myself!         |[190, 192]|1293753600    |


In [43]:
filtered = music.filter(music.overall>3.5).show(10)

+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0528881469| [0, 0]|    5.0|We got this GPS f...| 06 2, 2013| AO94DHGC771SJ|             amazdnu|     Gotta have GPS!|    1370131200|
|0594451647| [3, 3]|    5.0|I am using this w...| 01 3, 2014|A2JXAZZI9PHK9Z|Billy G. Noland "...|HDMI Nook adapter...|    1388707200|
|0594451647| [0, 0]|    5.0|This adaptor is r...| 05 4, 2014| AAZ084UMH8VZ2|D. L. Brown "A Kn...|A Perfdect Nook H...|    1399161600|
|0594451647| [0, 0]|    4.0|This adapter easi...|07 11, 2014| AEZ3CR6BKIROJ|        Mark Dietter|A nice easy to us...|    1405036800|
|0594451647| [3, 3]|    5.0|This product real...|01 20, 2014|A

In [46]:
music.drop('reviewerName')

DataFrame[asin: string, helpful: array<bigint>, overall: double, reviewText: string, reviewTime: string, reviewerID: string, summary: string, unixReviewTime: bigint]

In [74]:
music.unixReviewTime.alias('Unix Review Time')

Column<unixReviewTime AS `Unix Review Time`>

In [87]:
music.select('asin').groupby('asin').count().sort('count',ascending=False).show()

+----------+-----+
|      asin|count|
+----------+-----+
|B003VWJ2K8|  163|
|B0002E1G5C|  143|
|B0002F7K7Y|  116|
|B003VWKPHC|  114|
|B0002H0A3S|   93|
|B0002CZVXM|   74|
|B0006NDF8A|   71|
|B0009G1E0K|   69|
|B0002E2KPC|   68|
|B0002GLDQM|   67|
|B004XNK7AI|   65|
|B005FKF1PY|   63|
|B00646MZHK|   62|
|B0002GMGYA|   58|
|B001PGXHX0|   58|
|B0018TIADQ|   57|
|B000978D58|   57|
|B001PGXKC8|   55|
|B0002D0CEO|   55|
|B000068NW5|   52|
+----------+-----+
only showing top 20 rows



# Creating a DataFrame from JSON string (useful for Metadata.json)

In [55]:
jsonStrings = ['{"name":"Yin","address":{"city":"Columbus","state":"Ohio"}}','{"name":"Yon","address":{"city":"Columbus","state":"Ohio"}}']
otherPeopleRDD = sc.parallelize(jsonStrings)
otherPeople = spark.read.json(otherPeopleRDD)
otherPeople.show()

+---------------+----+
|        address|name|
+---------------+----+
|[Columbus,Ohio]| Yin|
|[Columbus,Ohio]| Yon|
+---------------+----+



In [61]:
metadata = spark.read.json('../../amazon_dataset/metadata.json')
metadata.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- imUrl: string (nullable = true)
 |-- price: double (nullable = true)
 |-- related: struct (nullable = true)
 |    |-- also_bought: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- also_viewed: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- bought_together: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- buy_after_viewing: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- salesRank: struct (nullable = true)
 |    |-- Appliances: long (nullable = true)
 |    |-- Arts, Crafts & Sewing: long (nullable = true)
 |    |-- Automotive: long

# IMPORTANT: Reviews Parsing and Querying

Using this function to properly parse each JSON dict:

In [1]:
import ujson
from toolz import dissoc
import datetime
import re
import time
from pyspark.sql.types import *

Function used to convert a UNIX timestamp to a proper datetime object (more human-readable):

In [2]:
def convert_date(date):
    date = datetime.date.fromtimestamp(int(date))
    return date

#Used to remove reviewerName (not interesting) and to convert UNIX timestamp in date object
def reviews_parser(line):
    line = ujson.loads(line)
    line['reviewTime'] = convert_date(line['unixReviewTime'])
    line['productID'] = line['asin']
    return dissoc(line,'reviewerName','unixReviewTime','asin')

In [3]:
f = '../../amazon_dataset/reviews_Digital_Music_5.json'

#Definition of schema for the DataFrame
string_fields = ['productID','reviewText','reviewerID','summary']
fields = [StructField(field_name, StringType(), False) for field_name in string_fields]
fields+=[StructField('reviewTime',DateType(),False),
         StructField('overall',FloatType(),False),
         StructField('helpful',ArrayType(IntegerType(),containsNull=False),False)]
schema = StructType(fields)

data = sc.textFile(f)
data = data.map(reviews_parser)
data = data.toDF(schema)

In [4]:
data.printSchema()

root
 |-- productID: string (nullable = false)
 |-- reviewText: string (nullable = false)
 |-- reviewerID: string (nullable = false)
 |-- summary: string (nullable = false)
 |-- reviewTime: date (nullable = false)
 |-- overall: float (nullable = false)
 |-- helpful: array (nullable = false)
 |    |-- element: integer (containsNull = false)



In [5]:
data.show(10)

+----------+--------------------+--------------+--------------------+----------+-------+--------+
| productID|          reviewText|    reviewerID|             summary|reviewTime|overall| helpful|
+----------+--------------------+--------------+--------------------+----------+-------+--------+
|5555991584|It's hard to beli...|A3EBHHCZO6V2A4|Enya's last great...|2006-09-12|    5.0|  [3, 3]|
|5555991584|A clasically-styl...| AZPWAXJG9OJXV|Enya at her most ...|2001-06-03|    5.0|  [0, 0]|
|5555991584|I never thought E...|A38IRL0X2T4DPF|     The best so far|2003-07-14|    5.0|  [2, 2]|
|5555991584|This is the third...|A22IK3I6U76GX0|Ireland produces ...|2000-05-03|    5.0|  [1, 1]|
|5555991584|Enya, despite bei...|A1AISPOIIHTHXX|4.5; music to dre...|2008-01-17|    4.0|  [1, 1]|
|5555991584|Who knows why I i...|A2P49WD75WHAG5|Dreamy unearthly ...|2003-07-30|    5.0|[62, 65]|
|5555991584|Enya is one of a ...|A3O90G1D7I5EGG|Have You Hugged A...|2000-05-27|    3.0|  [1, 5]|
|5555991584|Enya is 

In [6]:
data.registerTempTable('reviews')

### Trying to query along all the columns, to check if they work with their type:

**Date**: there are some problems, in the schema it's listed as 'date' type, instead it's impossible to extract year/month/day from the object (it raises exception); it works if it's treated as string object... WTF?

In [7]:
data.filter(data.reviewTime.startswith('2013-12-25')).show(10)

+----------+--------------------+--------------+--------------------+----------+-------+-------+
| productID|          reviewText|    reviewerID|             summary|reviewTime|overall|helpful|
+----------+--------------------+--------------+--------------------+----------+-------+-------+
|B000000NN1|I finally got thi...| AUHZ4NEDBKM1X|           a classic|2013-12-25|    5.0| [0, 0]|
|B00000J7OA|Nothing great or ...|A37Z81LW79DUZ8|Just another no l...|2013-12-25|    2.0| [0, 0]|
|B0050N8O7I|Its not bad, unti...|A2TJT5ULZFJ0YL|             Good...|2013-12-25|    3.0| [0, 0]|
|B005IXKMNO|I have always lov...|A2TJT5ULZFJ0YL|               RHCP!|2013-12-25|    4.0| [0, 0]|
|B00FN8H8XG|I downloaded this...|A2BFZD9WRHVO0J|      We Three Kings|2013-12-25|    3.0| [0, 0]|
|B00FW7T99E|I downloaded this...|A2BFZD9WRHVO0J|         Nutcracker.|2013-12-25|    2.0| [0, 0]|
+----------+--------------------+--------------+--------------------+----------+-------+-------+



Not working treating the reviewTime as a date object and trying to extract its attributes:

In [8]:
data.filter(data.reviewTime.year==2014).show(10)

AnalysisException: u"Can't extract value from reviewTime#4;"

Querying along **helpful**:

In [8]:
data.filter(data.helpful[1]-data.helpful[0]>20).show()

+----------+--------------------+--------------+--------------------+----------+-------+--------+
| productID|          reviewText|    reviewerID|             summary|reviewTime|overall| helpful|
+----------+--------------------+--------------+--------------------+----------+-------+--------+
|B00000016W|I never understoo...|A22N9H8V0RYQR3|      Overrated HYPE|2005-08-29|    1.0| [4, 29]|
|B00000016W|I am very disapoi...| AH1KJ45IQTM4T|        Masterpiece?|2003-11-28|    1.0|[18, 73]|
|B00000016W|I love the Beach ...|A1N643TZ48RPG7|More like the &qu...|2004-03-30|    1.0|[13, 65]|
|B0000005XM|Punk has had a lo...|A2Y4MUU6R9NDUR|NOFX meets &quot;...|2004-04-17|    1.0|[26, 66]|
|B00000064E|I know this guy i...|A1ZDDLHFSUGILJ|tired, aloof and coy|2000-05-21|    2.0| [1, 40]|
|B00000064F|This is so totaly...|A23YXA0AX47YPD|      UNOBNOXIOUS...|2003-06-25|    3.0| [2, 27]|
|B00000064G|I listen to alot ...|A1HUX7WN8E65R3| Sorry - but lame...|2006-11-20|    3.0| [3, 36]|
|B00000064G|Apparent

Querying along **productID**:

In [9]:
data.groupBy('productID').count().sort('count',ascending=False).show(10)

+----------+-----+
| productID|count|
+----------+-----+
|B0007NFL18|  272|
|B000084T18|  271|
|B00006690F|  204|
|B00004T9UF|  202|
|B00005YW4H|  190|
|B0006ZQ9BS|  185|
|B0000AGWFX|  181|
|B00005O54Q|  169|
|B00000163G|  166|
|B00065XJ52|  159|
+----------+-----+
only showing top 10 rows



Querying along **reviewerID**:

In [4]:
data.groupBy('reviewerID').count().withColumnRenamed("count", "total").sort('total',ascending=False).filter('total%2==0').show(10)

+--------------+-----+
|    reviewerID|total|
+--------------+-----+
| A9Q28YTLYREO7|  578|
| A8IFUOL8S9BZC|  256|
|A1GN8UJIZLCA59|  246|
|A200C7YQJ45LRR|  200|
|A1J5KCZC8CMW9I|  182|
| A8SCX6VUTE05H|  172|
|A2582KMXLK2P06|  160|
|A1IKOYZVFHO1XP|  144|
|A12R54MKO17TW0|  132|
|A2ETZ7GF5B1712|  130|
+--------------+-----+
only showing top 10 rows

